##### Importing required packages and libraries 

In [1]:
import scipy.io as sio
from scipy.sparse import csr_matrix 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

In [2]:
import itertools
import warnings
from pyparsing import null_debug_action
from cmath import nan

In [3]:
#plotly
import plotly
import plotly.graph_objects as go

In [4]:
#networkx
import networkx as nx
from networkx.algorithms import community
import holoviews as hv
from holoviews import opts

In [5]:
#bokeh for plotting
import bokeh.plotting
import bokeh.plotting as bk_plotting
from bokeh.plotting import from_networkx, figure
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges, ColorBar
from bokeh.io import output_notebook, save, show, reset_output
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges
import bokeh.models as bk_models
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8, Spectral11, Category20_15, Category20_20, Turbo256
from bokeh.transform import linear_cmap
from bokeh.io import output_file, show
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool, PanTool,  WheelZoomTool, ResetTool, SaveTool, BoxZoomTool)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx

##### Loading .mat files - respPeak = peak of ind, allROIPositions

In [6]:
mat_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/respPeak.mat')['x']
roi_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/allROIPositions.mat')['allROIPositions']
list1 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/indicesSession1.mat')['indSess1']
list2 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/List2.mat')['List2']
list3 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/List3.mat')['List3']
list4 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/List4.mat')['List4']
lista = [[k[0] for k in list1], [k[0] for k in list2], [k[0] for k in list3], [k[0] for k in list4]]
ind_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/blFinalHit.mat')['blFinalHit']
indices_load = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/indicesSession1.mat')['indSess1']

In [7]:
tune1 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/tuneCat1.mat')['tuneCat1']
tune2 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/tuneCat2.mat')['tuneCat2']
tune3 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/tuneCat3.mat')['tuneCat3']
tune4 = sio.loadmat('C:/Users/m.nedeljkovic/Desktop/tuneCat4.mat')['tuneCat4']
tune = [tune1, tune2, tune3, tune4]

##### General functions

In [8]:
#Dividing data into trials, anst state and awake state

def divide_data(index, trial, mat_load, ind_load):

    index[trial] = index[trial] + index[trial-1]
    dat_tr = mat_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]
    ind_tr = ind_load[index[trial-1] : index[trial]+1, :, :, 0, :, :]

    #choose state
    dat_stan = dat_tr[:, 0, :, :, :]
    dat_staw = dat_tr[:, 1, :, :, :]
    ind_stan = ind_tr[:, 0, :, :, :]
    ind_staw = ind_tr[:, 1, :, :, :]
    
    return index, dat_tr, dat_stan, dat_staw, ind_stan, ind_staw

In [9]:
#make dict to convert to dataframe and computing correlation

def make_dict(index, trial, lista, dat_sean, dat_seaw, ind_sean, ind_seaw):

    dat_df_an = []
    dat_df_aw = []
    ind_df_an = []
    ind_df_aw = []
    for matrix in range(0,np.shape(dat_sean)[0]-1):
        if(index[trial-1]+matrix in lista):
            dat_df_an.append(dat_sean[matrix].flatten())
            dat_df_aw.append(dat_seaw[matrix].flatten())
            ind_df_an.append(ind_sean[matrix].flatten())
            ind_df_aw.append(ind_seaw[matrix].flatten())
    dat_df_an = pd.DataFrame(dat_df_an)*pd.DataFrame(ind_df_an)
    dat_df_aw = pd.DataFrame(dat_df_aw)*pd.DataFrame(ind_df_aw)
    dat_df_an = dat_df_an.T.corr()
    dat_df_aw = dat_df_aw.T.corr()
    return dat_df_an, dat_df_aw

def make_dict_one_speaker(index, trial, lista, dat_sean, dat_seaw, num, ind_sean, ind_seaw):
    
    dat_df_an = []
    dat_df_aw = []
    ind_df_an = []
    ind_df_aw = []
    for matrix in range(0,np.shape(dat_sean)[0]-1):
        if(index[trial-1]+matrix in lista):
            dat_df_an.append(dat_sean[matrix][num])
            dat_df_aw.append(dat_seaw[matrix][num])
            ind_df_an.append(ind_sean[matrix][num])
            ind_df_aw.append(ind_seaw[matrix][num])
    dat_df_an = pd.DataFrame(dat_df_an)*pd.DataFrame(ind_df_an)
    dat_df_aw = pd.DataFrame(dat_df_aw)*pd.DataFrame(ind_df_aw)

    dat_df_an = pd.DataFrame(dat_df_an).T.corr()
    dat_df_aw = pd.DataFrame(dat_df_aw).T.corr()
    return dat_df_an, dat_df_aw

In [10]:
#exclude the rows/columns which index is not in the other dataframe
def exclude_nan(dat_df_an, dat_df_aw):

    df_corr_an = dat_df_an.dropna(how='all', axis=1).dropna(how='all', axis=0)
    df_corr_aw = dat_df_aw.dropna(how='all', axis=1).dropna(how='all', axis=0)

    mask = df_corr_aw.index.isin(df_corr_an.index)
    df_corr_aw = df_corr_aw.loc[mask]
    mask = df_corr_an.index.isin(df_corr_aw.index)
    df_corr_an = df_corr_an.loc[mask]
    mask = []
    for x in df_corr_an.columns:
        if x not in df_corr_aw.columns:
            mask.append(x)
    df_corr_an = df_corr_an.drop(columns=mask)
    mask = []
    for x in df_corr_aw.columns:
        if x not in df_corr_an.columns:
            mask.append(x)
    df_corr_aw = df_corr_aw.drop(columns=mask)

    return df_corr_an, df_corr_aw

#### Plotting network graph

##### Threshold

In [152]:
threshold = 0.6

##### Form_links

In [12]:
def form_links(df_corr, df_pos):

    x_coor = df_pos[0]
    y_coor = df_pos[1]
    links = df_corr.stack()
    links = links.reset_index()
    links.columns = ["var1", "var2", "value"]
    links.loc[:, "x0"] = [0 for i in range(len(links))]
    links.loc[:, "y0"] = [0 for i in range(len(links))]
    links.loc[:, "x1"] = [0 for i in range(len(links))]
    links.loc[:, "y1"] = [0 for i in range(len(links))]

    for i in range(len(links['x1'])):
        links['x0'][i] = x_coor[int(links['var1'][i])]
        links['x1'][i] = x_coor[int(links['var2'][i])]
        links['y0'][i] = y_coor[int(links['var1'][i])]
        links['y1'][i] = y_coor[int(links['var2'][i])]
    
    return links

In [81]:
def form_links(df_corr, df_pos, df_tuned):

    x_coor = df_pos[0]
    y_coor = df_pos[1]
    links = df_corr.stack()
    links = links.reset_index()
    
    links.columns = ["var1", "var2", "value"]
    tunes = []
    j = links['var1'][0]
    br = 0
    for i in links['var1']:
        if(i!=j):
            br+=1
        tunes.append(df_tuned[br])
        j = i
    links['tunes'] = tunes
    links.columns = ["var1", "var2", "value", "tunes"]

    links.loc[:, "x0"] = [0 for i in range(len(links))]
    links.loc[:, "y0"] = [0 for i in range(len(links))]
    links.loc[:, "x1"] = [0 for i in range(len(links))]
    links.loc[:, "y1"] = [0 for i in range(len(links))]

    for i in range(len(links['x1'])):
        links['x0'][i] = x_coor[int(links['var1'][i])]
        links['x1'][i] = x_coor[int(links['var2'][i])]
        links['y0'][i] = y_coor[int(links['var1'][i])]
        links['y1'][i] = y_coor[int(links['var2'][i])]
    
    return links

##### Initial_graph

In [147]:
def initial_graph(links):
    
    G=nx.from_pandas_edgelist(links, source="var1", target="var2", edge_attr=True)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.edges[edge]['x0'], G.edges[edge]['y0']
        x1, y1 = G.edges[edge]['x1'], G.edges[edge]['y1']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    return G, edge_x, edge_y, edge_trace

##### Data_management

In [155]:
def data_management(links, G):
    
    mask = ~links['var1'].duplicated()
    # Get a list of nodes with attributes
    nodes = links[mask][["var1", "x0","var2","x1"]]
    attr_dict_x = nodes.set_index('var1')['x0'].to_dict()
    nx.set_node_attributes(G, attr_dict_x, 'x0')

    mask = ~links['var1'].duplicated()
    # Get a list of nodes with attributes
    nodes = links[mask][["var1", "y0","var2","y1"]]
    attr_dict_y = nodes.set_index('var1')['y0'].to_dict()
    nx.set_node_attributes(G, attr_dict_y, 'y0')

    mask = ~links['var1'].duplicated()
    # Get a list of nodes with attributes
    nodes = links[mask][["var1", "tunes"]]
    attr_dict_t = nodes.set_index('var1')['tunes'].to_dict()
    nx.set_node_attributes(G, attr_dict_t, 'tunes')

    edge_attrs = {}
    
    for start_node, end_node, info in G.edges(data=True):
        edge_attrs[(start_node, end_node)] = info['value']
        
            
    nx.set_edge_attributes(G, edge_attrs, "edge_value")
    
    tunes = 'tunes'

    HOVER_TOOLTIPS = [
        ("index", "$index"),
        ("x", "@x0"),
        ("y", "@y0"),
        ("value", "@edge_value"),
        ("tunes", "@tunes")
    ]

    return G, HOVER_TOOLTIPS, tunes, attr_dict_x, attr_dict_y

##### Node_size

In [90]:
def node_size(G):
    
    adjusted_node_size = dict([(node, 10) for node, degree in nx.degree(G)])
    nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
    size_by_this_attribute = 'adjusted_node_size'
    
    return G, size_by_this_attribute
    

##### Remove_edges

In [98]:
def remove_edges(G, threshold):
    
    G.remove_edges_from([(x, y) for x, y, t in G.edges.data(data=True) if t['value'] < threshold and t['value'] > -threshold])
    
    return G

##### Plot_graph

In [168]:
def plot_graph(attr_dict_x, attr_dict_y, G, edge_trace, size_by_this_attribute, tunes, HOVER_TOOLTIPS):

    plot = Plot(width=1000, height=720,
            x_range=Range1d(min(attr_dict_x)*4-40, max(attr_dict_x)*4+40), y_range=Range1d(min(attr_dict_y)*4-40, max(attr_dict_y)*4+40))
    plot.add_tools(HoverTool(tooltips=HOVER_TOOLTIPS), TapTool(), BoxSelectTool(), WheelZoomTool(), ResetTool(), PanTool(), SaveTool(), BoxZoomTool())
    pos = {}
    for i in nx.spring_layout(G, iterations=40).keys():
        pos[i] = np.array([attr_dict_x[i], attr_dict_y[i]])
    network_graph = from_networkx(G, pos, scale = 1, center = (0, 0))

    #Set node sizes and colors according to node degree (color as category from attribute)
    #network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute)

    #Set edge opacity and width
    #network_graph.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.5, line_width=1)
    minimum_value_color = 1
    maximum_value_color = 6
    mapper1 = linear_cmap(field_name='tunes', palette = Purples8, low=minimum_value_color, high=maximum_value_color)
    
    network_graph.node_renderer.glyph = Circle(size=15, fill_color=mapper1)
    network_graph.node_renderer.selection_glyph = Circle(size=15, fill_color=mapper1)
    network_graph.node_renderer.hover_glyph = Circle(size=15, fill_color=mapper1)

    min_val = -1
    max_val = 1
   
    mapper = linear_cmap(field_name='value', palette=Turbo256 ,low=min_val ,high=max_val)

    network_graph.edge_renderer.glyph = MultiLine(line_color=mapper, line_alpha=0.5, line_width=5)
    network_graph.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[0], line_width=5)
    network_graph.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[0], line_width=5)


    network_graph.selection_policy = NodesAndLinkedEdges()
    network_graph.inspection_policy = EdgesAndLinkedNodes()

    plot.renderers.append(network_graph)

    color_bar = ColorBar(color_mapper=mapper['transform'], width=8)
    color_bar1 = ColorBar(color_mapper=mapper1['transform'], width=8)
    plot.add_layout(color_bar1, 'left')
    plot.add_layout(color_bar, 'right')
    
    show(plot)

##### Plot_network_graph

In [53]:
def plot_network_graph(links):
    
    G, edge_x, edge_y, edge_trace = initial_graph(links)
    G, HOVER_TOOLTIPS, color_by_this_attribute, attr_dict_x, attr_dict_y = data_management(links, G)
    G, size_by_this_attribute = node_size(G)
    G = remove_edges(G, threshold)
    output_notebook()
    plot_graph(attr_dict_x, attr_dict_y, G, edge_trace, size_by_this_attribute, color_by_this_attribute, HOVER_TOOLTIPS)

In [20]:
"""
Different ways to plot the network graph

plt.figure(figsize=(15,15))
for i in range(0, len(y_coor), 1):
    plt.plot(x_coor[i:i+2], y_coor[i:i+2], 'ro-')
plt.plot(df_an_pos[0], df_an_pos[1], 'o')
"""

"\nDifferent ways to plot the network graph\n\nplt.figure(figsize=(15,15))\nfor i in range(0, len(y_coor), 1):\n    plt.plot(x_coor[i:i+2], y_coor[i:i+2], 'ro-')\nplt.plot(df_an_pos[0], df_an_pos[1], 'o')\n"

### Graph representation

In [178]:
warnings.filterwarnings('ignore')

#lenghts of each trial
index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]
start_an = 0
start_aw = 0

for trial in range(1,len(index)):
    
    index, dat_tr, dat_stan, dat_staw, ind_stan, ind_staw = divide_data(index, trial, mat_load, ind_load)
    pos_tr = roi_load[index[trial-1] : index[trial]+1, :, :, :]

    #choose state
    pos_stan = pos_tr[:, :, 0, :]
    pos_staw = pos_tr[:, :, 1, :]
    
    for sesion in range(4):

        tune_ses = tune[sesion][:, :, 0]

        #choose session
        dat_sean = dat_stan[:, sesion, :, :]
        dat_seaw = dat_staw[:, sesion, :, :]
        pos_sean = pos_stan[:, :, sesion]
        pos_seaw = pos_staw[:, :, sesion]
        ind_sean = ind_stan[:, sesion, :, :]
        ind_seaw = ind_staw[:, sesion, :, :]
        tune_an = tune_ses[:, 0]
        tune_aw = tune_ses[:, 1]

        #make dict to convert to dataframe
        dat_df_an, dat_df_aw = make_dict(index, trial, lista[sesion], dat_sean, dat_seaw, ind_sean, ind_seaw)

        #exclude the rows/columns which index is not in the other dataframe
        #df_corr_an, df_corr_aw = exclude_nan(dat_df_an, dat_df_aw)
        df_corr_an, df_corr_aw = dat_df_an.dropna(axis=1, how='all').dropna(axis=0, how='all'), dat_df_aw.dropna(axis=1, how='all').dropna(axis=0, how='all')
        tune_an, tune_aw = tune_an[start_an : start_an+len(df_corr_an)], tune_aw[start_aw : start_aw+len(df_corr_aw)]
        start_an += len(df_corr_an)
        start_aw += len(df_corr_aw)

        df_corr_an_pos = pd.DataFrame(index=df_corr_an.index, columns=df_corr_an.columns)
        df_corr_aw_pos = pd.DataFrame(index=df_corr_aw.index, columns=df_corr_aw.columns)
        
        for i in df_corr_an.index:
            for j in df_corr_an.columns:
                df_corr_an_pos.loc[i, j] = np.sqrt((pos_sean[i, 0]-pos_sean[j,0])**2 + (pos_sean[i, 1]-pos_sean[j,1])**2)
                df_corr_aw_pos.loc[i, j] = np.sqrt((pos_seaw[i, 0]-pos_seaw[j,0])**2 + (pos_seaw[i, 1]-pos_seaw[j,1])**2)

        df_an_pos = pd.DataFrame(df_corr_an_pos, index=None).astype(float)
        df_aw_pos = pd.DataFrame(df_corr_aw_pos, index=None).astype(float)

        #distance correlation
        df_corr_an_pos = df_an_pos.corr().dropna(how='all', axis=1).dropna(how='all', axis=0)
        df_corr_aw_pos = df_aw_pos.corr().dropna(how='all', axis=1).dropna(how='all', axis=0)

        #df_corr_diff = (df_corr_aw-df_corr_an)
        links = form_links(df_corr_an, df_an_pos, tune_an)
        plot_network_graph(links)
    
    break               
    

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [181]:
warnings.filterwarnings('ignore')

#lenghts of each trial
index = [0, 118, 88, 100, 98, 109, 78, 121, 76, 113, 97, 112, 118]
start_an = 0
start_aw = 0

for trial in range(1,len(index)):
    
    index, dat_tr, dat_stan, dat_staw, ind_stan, ind_staw = divide_data(index, trial, mat_load, ind_load)
    pos_tr = roi_load[index[trial-1] : index[trial]+1, :, :, :]

    #choose state
    pos_stan = pos_tr[:, :, 0, :]
    pos_staw = pos_tr[:, :, 1, :]
    
    for sesion in range(4):

        tune_ses = tune[sesion][:, :, 0]

        #choose session
        dat_sean = dat_stan[:, sesion, :, :]
        dat_seaw = dat_staw[:, sesion, :, :]
        pos_sean = pos_stan[:, :, sesion]
        pos_seaw = pos_staw[:, :, sesion]
        ind_sean = ind_stan[:, sesion, :, :]
        ind_seaw = ind_staw[:, sesion, :, :]
        tune_an = tune_ses[:, 0]
        tune_aw = tune_ses[:, 1]

        #make dict to convert to dataframe
        dat_df_an, dat_df_aw = make_dict(index, trial, lista[sesion], dat_sean, dat_seaw, ind_sean, ind_seaw)

        #exclude the rows/columns which index is not in the other dataframe
        #df_corr_an, df_corr_aw = exclude_nan(dat_df_an, dat_df_aw)
        df_corr_an, df_corr_aw = dat_df_an.dropna(axis=1, how='all').dropna(axis=0, how='all'), dat_df_aw.dropna(axis=1, how='all').dropna(axis=0, how='all')
        tune_an, tune_aw = tune_an[start_an : start_an+len(df_corr_an)], tune_aw[start_aw : start_aw+len(df_corr_aw)]
        start_an += len(df_corr_an)
        start_aw += len(df_corr_aw)

        df_corr_an_pos = pd.DataFrame(index=df_corr_an.index, columns=df_corr_an.columns)
        df_corr_aw_pos = pd.DataFrame(index=df_corr_aw.index, columns=df_corr_aw.columns)
        
        for i in df_corr_an.index:
            for j in df_corr_an.columns:
                df_corr_an_pos.loc[i, j] = np.sqrt((pos_sean[i, 0]-pos_sean[j,0])**2 + (pos_sean[i, 1]-pos_sean[j,1])**2)
                df_corr_aw_pos.loc[i, j] = np.sqrt((pos_seaw[i, 0]-pos_seaw[j,0])**2 + (pos_seaw[i, 1]-pos_seaw[j,1])**2)

        df_an_pos = pd.DataFrame(df_corr_an_pos, index=None).astype(float)
        df_aw_pos = pd.DataFrame(df_corr_aw_pos, index=None).astype(float)

        #distance correlation
        df_corr_an_pos = df_an_pos.corr().dropna(how='all', axis=1).dropna(how='all', axis=0)
        df_corr_aw_pos = df_aw_pos.corr().dropna(how='all', axis=1).dropna(how='all', axis=0)

        #df_corr_diff = (df_corr_aw-df_corr_an)
        links = form_links(df_corr_aw, df_aw_pos, tune_aw)
        plot_network_graph(links)
        
    break    

Loading BokehJS ...